# ADE Visualization example

Any Cloud Optimized GeoTIFF(COG) in an ADE workspace can be visualized in a dynamic map by using a tiling service hosted in MAAP.

Steps:
1. Make a list of TIFFs in your workspace to use as a single layer
2. Generate a MosaicJson file from this list of files (or a GeoJson index)
3. Combine the MosaicJson with other tiler visualization parameters to register a layer with your visualization tool.



In [ ]:
# Package library requirement needed on new workspace
#%conda install -c conda-forge cogeo-mosaic
!pip install cogeo-mosaic

In [13]:
import glob
import os

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

## Build a list of files

You can either make a list of file paths, or create a geojson layer with a column containing the file paths.
The paths need to be **s3** paths currently.

In [15]:
# Local Path to your COGs
dps_output = "/projects/maap-users/alexdevseed/landsat8/viz"

# Search for files to include, use recursive if nested folders (common in DPS output)
files = glob.glob(os.path.join(dps_output, "Landsat*_dps.tif"), recursive=False)

def local_to_s3(url):
    return url.replace("/projects/maap-users", "s3://maap-ops-dataset/maap-users")

tiles = [local_to_s3(file) for file in files]
print(tiles)

['s3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30542_comp_cog_2015-2020_dps.tif',
 's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30543_comp_cog_2015-2020_dps.tif',
 's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30822_comp_cog_2015-2020_dps.tif',
 's3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/Landsat8_30823_comp_cog_2015-2020_dps.tif']

### How to find the s3 path

You might be wondering how to find the s3 path to begin with. Right now the easiest way is to right click on a file in the file explorer on the left panel, and Get Presigned S3 Url.

It will look something like `https://maap-ops-dataset.s3.amazonaws.com/maap-users/alexdevseed/landsat8/viz/Landsat8_30542_comp_cog_2015-2020_dps.tif?AWSAccessKeyId...`

The first part of the url is the bucket name: `maap-ops-dataset`
After the next `/` it then matches to the local path.

Future versions of MAAP will include functions to do this part for you...

## Make a mosaic

In [4]:
# Now take the list of s3 paths an generate a mosaic
# TODO: if you have a lot of files (more than 100), creating a geojson index and using from_features will be more efficient.

mosaicdata = MosaicJSON.from_urls(tiles, minzoom=9, maxzoom=16)


In [5]:
# Now save the mosaicjson to a file on S3 so the tile server can use it.
with MosaicBackend("/projects/maap-users/alexdevseed/landsat8/viz/L8_tile_test.json", mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)

# The s3 path to the file is going to be
# s3://maap-ops-dataset/maap-users/alexdevseed/L8_tile_test.json

## Make a Map

In [1]:
import ipycmc

In [7]:
w = ipycmc.MapCMC()
w

MapCMC()

In [8]:
wmts_url = "https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml"
params = [
    "tile_format=png",
    "tile_scale=2",
    "pixel_selection=first",
    "TileMatrixSetId=WebMercatorQuad",
    "url=s3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/L8_tile_test.json",
    "bidx=6",
    "resampling_method=nearest",
    "rescale=0%2C1",
    "return_mask=true",
    "colormap_name=viridis"
]
#TODO url encode so the params can be human readable
wmts_call = "?".join([wmts_url, "&".join([*params])])
wmts_call

'https://baxpil3vd6.execute-api.us-east-1.amazonaws.com/mosaicjson/WMTSCapabilities.xml?tile_format=png&tile_scale=2&pixel_selection=first&TileMatrixSetId=WebMercatorQuad&url=s3://maap-ops-dataset/maap-users/alexdevseed/landsat8/viz/L8_tile_test.json&bidx=6&resampling_method=nearest&rescale=0%2C1&return_mask=true&colormap_name=viridis'

In [9]:
# This adds a new layer to the map above, call Cloud Optimized GeoTIFF
w.load_layer_config(wmts_call
                    , "wmts/xml")